### Connecting to SQL Server with Python and pyodbc

A complete tutorial using `pyodbc` and `pandas` to work with SQL Server.

---

You can choose wether you'd like to create a new environment for this or not.

If you do, do so now.

---

In [ ]:
#install dependencies pyodbc (dont forget ipython ipykernel if you are using jupyter)

!pip install pyodbc pandas

*Note*: You might have to install ODBC Driver 17, or greater, for SQL. 

You can find it [here](https://learn.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver16).

---

### Step 1: Connect to SQL Server using pyodbc

In [ ]:
import pyodbc

server_name = 'LAPTOP-OM16N5V6'  # replace with your server name
database_name = 'AI24'           # replace with your database name

# Define connection string
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=localhost;'
    f'SERVER={server_name};'
    f'DATABASE={database_name};'
    'Trusted_Connection=yes;'
)

print("Connected!")

### Step 2: Create Schema & Tables

In [ ]:
cursor = conn.cursor()

In [ ]:
# Create schema

cursor.execute('CREATE SCHEMA Library;')

conn.commit()   # Save changes to database

In [ ]:
# Create tables
cursor.execute('''
                  CREATE TABLE Library.Authors (
                      AuthorID INT PRIMARY KEY IDENTITY,
                      FirstName NVARCHAR(100) NOT NULL,
                      LastName NVARCHAR(100) NOT NULL
              );''')

cursor.execute('''
                  CREATE TABLE Library.Books (
                        BookID INT PRIMARY KEY IDENTITY,
                        Title NVARCHAR(255) NOT NULL,
                        Genre NVARCHAR(50),
                        AuthorID INT FOREIGN KEY REFERENCES Library.Authors(AuthorID)
                      );''')

conn.commit()   # Save changes to database

🔁 conn.commit() — What It Does:

conn.commit() saves all changes made during the current database transaction.

Whenever you:

    INSERT

    UPDATE

    DELETE

    or make any other change to the data or schema

...those changes are not permanent until you call commit().

🔁 When You Don’t Need It:

    For read-only queries (SELECT), you don’t need commit().

### Step 3: Insert Sample Data

In [ ]:
cursor.execute("INSERT INTO Library.Authors (FirstName, LastName) VALUES (?, ?);", ('Jane', 'Austen'))        # method 1, for single inserts

conn.commit()

In [ ]:
data_for_authors = [('Harper', 'Lee'), ('J.R.R.', 'Tolkien')]
data_for_books = [('Pride and Prejudice', 'Classic', 1), ('To Kill a Mockingbird', 'Fiction', 2), ('The Hobbit', 'Fantasy', 3)]

In [ ]:
cursor.executemany("INSERT INTO Library.Authors (FirstName, LastName) VALUES (?, ?);", data_for_authors)      # method 2, for bulk inserts


cursor.executemany("INSERT INTO Library.Books (Title, Genre, AuthorID) VALUES (?, ?, ?);", data_for_books)

conn.commit()

### Step 4: Query Data using pyodbc

In [ ]:
cursor.execute("SELECT Title, Genre FROM Library.Books;")

for row in cursor.fetchall():
    print(row)

### Step 5: Load Data into Pandas DataFrame

In [ ]:
import pandas as pd

df = pd.read_sql("""SELECT 
                        b.Title, 
                        a.FirstName, 
                        a.LastName 
                    FROM 
                        library.Books b 
                        JOIN library.Authors a ON b.AuthorID = a.AuthorID;"""
                 , conn);

df

### pyodbc documentation

There are many more actions you can take via pyodbc. For more information, check out the [pyodbc documentation](https://github.com/mkleehammer/pyodbc/wiki).